# CNN classification
- Classification is done for 3 shapes: circles, squares and triangles.
- There are 100 images in each class.
- Each image is of size 28$\times$28 with 1 channel.
- Dataset contains images as img_n.png where (n mod 3) == (0, 1, 2) correspond to (circles, squares, triangles).
- Train-test will have 80-20 spilt giving 240 images to train and 60 to test.
- Images will be processes in batches of 30.
- **Result**: Accuracy of 81% was achieved. This shows that this model is learning since a random model would predict with 33% accuracy.

## Imports

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import cv2

## Data loader

In [0]:
def give_labels():
  labels = []
  for i in range(30):
    labels.append(i%3)
  return torch.Tensor(labels).long()
  
def give_train_data():
  data = []
  batch = []
  labels = give_labels()

  for i in range(240):
    filename = 'data/img_' + str(i) + '.png'
    batch.append(cv2.imread(filename, 0))
    if i % 30 == 29:
      batch = np.expand_dims(np.array(batch), 1)
      data.append(((torch.from_numpy(batch)).float(), labels))
      batch = []
  return data

def give_test_data():
  data = []
  batch = []
  labels = give_labels()

  for i in range(240, 300):
    filename = 'data/img_' + str(i) + '.png'
    batch.append(cv2.imread(filename, 0))
    if i % 30 == 29:
      batch = np.expand_dims(np.array(batch), 1)
      data.append(((torch.from_numpy(batch)).float(), labels))
      batch = []
  return data

## Defining network

In [0]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution kernel
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, stride=1, padding=1)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(8 * 7 * 7, 128)  # 7*7 from image dimension
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 3)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x), dim=1)
        # x = F.softmax(self.fc3(x), dim=1)
        return x

net = Net()

## Training

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [239]:
for epoch in range(40):  # loop over the dataset multiple times
    rloss = 0
    for i, data in enumerate(give_train_data(), 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        rloss += loss.item()

    print('[%d] loss: %.3f' % (epoch + 1, rloss/8))
        
print('Finished Training')

[1] loss: 1.170
[2] loss: 1.104
[3] loss: 1.101
[4] loss: 1.097
[5] loss: 1.091
[6] loss: 1.086
[7] loss: 1.079
[8] loss: 1.071
[9] loss: 1.060
[10] loss: 1.049
[11] loss: 1.031
[12] loss: 1.012
[13] loss: 0.996
[14] loss: 0.980
[15] loss: 0.939
[16] loss: 0.891
[17] loss: 0.825
[18] loss: 0.740
[19] loss: 0.823
[20] loss: 0.825
[21] loss: 1.086
[22] loss: 0.807
[23] loss: 0.657
[24] loss: 0.549
[25] loss: 0.466
[26] loss: 0.403
[27] loss: 0.343
[28] loss: 0.280
[29] loss: 0.231
[30] loss: 0.190
[31] loss: 0.143
[32] loss: 0.107
[33] loss: 0.080
[34] loss: 0.070
[35] loss: 0.049
[36] loss: 0.037
[37] loss: 0.031
[38] loss: 0.024
[39] loss: 0.020
[40] loss: 0.017
Finished Training


## Testing

In [240]:
correct = 0
total = 0
with torch.no_grad():
    for data in give_test_data():
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 60 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 60 test images: 81 %
